In [1]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np

from models.random_forest import software

In [2]:
#---
# Initialize
#---
model_run = "rf017"
run_id = 0
season = "winter"
detrend_type = "linear" # "constant", "linear"
predictors_of_model = [ # Note!: If prefilling "pf" is used, they have to be always stated after ERA5 predictors
    ["sp", "u10"],
    ["pf",],
    ["tp", "tp", "tp", "u10",],
    ["u10", "pf", "pf", "pf",],
    ["u10", "u10", "pf"],
]
timelags_of_model = [ # Note!: If pf is used alone as a predictor, timelags need to be in hours instead of days
    # [0, 2, 7], 
    # [7 * 24,]
    [3, 2],
    [7 * 24,],
    [7, 5, 2, 2,],
    [3, 7, 5, 2,],
    [5, 2, 7,],

]
models_path = [
]
percentile = 0.95
clf = RandomForestClassifier

# Build Hyperparameter Grid to optimize from.
#---
n_estimators = [int(x) for x in np.linspace(start = 0, stop = 1000, num = 4)] # Number of trees in random forest
max_depth = [int(x) for x in np.linspace(1, 3, num = 3)] # Maximum number of levels in tree

# max_depth.append(None)
# min_samples_split = [2, 5,] # Minimum number of samples required to split a node
# min_samples_leaf = [1, 2, 4] # Minimum number of samples required at each leaf node
random_state = 0

hparam_grid = {'n_estimators': n_estimators, # hparam grid if optimization is needed
            'max_depth': max_depth,
            # 'min_samples_split': min_samples_split,
            # 'min_samples_leaf': min_samples_leaf,
            'criterion' : ['gini',],
            'random_state' : [random_state,], # To compare results when changing hyperparameters
            'class_weight' : ["balanced",], # “balanced_subsample"
            'oob_score' : [True,],
            }

optimizer = "RandomSearchCV" # "RandomSearchCV", "GridSearchCV"
k = 3
n_iter = 100 
test_size = 0.25
is_optimized = True 
is_scaled = True
is_overlay_importance = True
is_station_name = False 

In [3]:
station_names_of_model = [
    ["hanko-han-fin-cmems",], # Finland
    ["forsmark-for-swe-cmems",], # West-Sweden
    ["kalixstoron-kal-swe-cmems",], # North-Sweden
    ["hamina-ham-fin-cmems",], # Finland deep in bay, "close" to russia, >1971
    ["oskarshamn-osk-swe-cmems"], # West-Sweden (South) < 2000 not necessarily need this
    # ["daugavgriva-dau-lva-cmems",], # Lettland (Riga) 2005 - 2020 make sure this works
    # ["travemuende-tra-deu-cmems",], # Germany (Travemuende) 2005 - 2020 make sure this works
]
model_run_flag = [
    "FIN", 
    "WSWE", 
    "NSWE", 
    "FINBAY", 
    "WSWE2",
    # "LVA", 
    # "DEU",]
]
for i, station_names in enumerate(station_names_of_model):
    model_run_code = f"{model_run}_{model_run_flag[i]}"
    print(model_run_code)
    

rf017_FIN
rf017_WSWE
rf017_NSWE
rf017_FINBAY
rf017_WSWE2


In [4]:
for i, station_names in enumerate(station_names_of_model):
    model_run_code = f"{model_run}_{model_run_flag[i]}"
    print(model_run_code)
    software.run(
model_run_code, 
run_id, 
season,
station_names,
detrend_type, 
predictors_of_model, 
timelags_of_model, 
models_path, 
percentile, 
clf, 
hparam_grid, 
optimizer, 
k, 
n_iter, 
random_state,
test_size,
is_optimized, 
is_scaled,
is_overlay_importance,
is_station_name,
)

# Shutdown pc
import os
os.system("shutdown /s /t 1") 

rf017_FIN
Load ERA5-Predictor: sp in region: lon-0530_lat7040 for years: 1999-2008 in season: winter
Load Predictand from GESLA
Applied one-hot-encoding with Percentile: 0.95


d:\Programme\miniconda3\envs\mlpy38\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 12 is smaller than n_iter=100. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
d:\Programme\miniconda3\envs\mlpy38\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
9 fits failed out of a total of 36.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "d:\Programme\miniconda3\envs\mlpy38\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\Programme\miniconda3\envs\ml

rf017_WSWE
Load ERA5-Predictor: sp in region: lon-0530_lat7040 for years: 1999-2008 in season: winter
Load Predictand from GESLA
Applied one-hot-encoding with Percentile: 0.95
Created directory: results/random_forest/rf017_WSWE/


d:\Programme\miniconda3\envs\mlpy38\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 12 is smaller than n_iter=100. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
d:\Programme\miniconda3\envs\mlpy38\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
9 fits failed out of a total of 36.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "d:\Programme\miniconda3\envs\mlpy38\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\Programme\miniconda3\envs\ml

rf017_NSWE
Load ERA5-Predictor: sp in region: lon-0530_lat7040 for years: 1999-2008 in season: winter
Load Predictand from GESLA
Applied one-hot-encoding with Percentile: 0.95
Created directory: results/random_forest/rf017_NSWE/


...

rf017_FINBAY
Load ERA5-Predictor: sp in region: lon-0530_lat7040 for years: 1999-2008 in season: winter
Load Predictand from GESLA
Applied one-hot-encoding with Percentile: 0.95
Created directory: results/random_forest/rf017_FINBAY/


d:\Programme\miniconda3\envs\mlpy38\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 12 is smaller than n_iter=100. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
d:\Programme\miniconda3\envs\mlpy38\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
9 fits failed out of a total of 36.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "d:\Programme\miniconda3\envs\mlpy38\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\Programme\miniconda3\envs\ml

rf017_WSWE2
Load ERA5-Predictor: sp in region: lon-0530_lat7040 for years: 1999-2008 in season: winter
Load Predictand from GESLA
Applied one-hot-encoding with Percentile: 0.95
Created directory: results/random_forest/rf017_WSWE2/


d:\Programme\miniconda3\envs\mlpy38\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 12 is smaller than n_iter=100. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
d:\Programme\miniconda3\envs\mlpy38\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
9 fits failed out of a total of 36.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "d:\Programme\miniconda3\envs\mlpy38\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\Programme\miniconda3\envs\ml

0